In [2]:
import pandas as pd
import numpy as np
from Lib import Algen_lib as lib

# Load Data

In [3]:
#Data Latih
df = pd.read_excel('data/sms_clean_norm_+_new_data_3.xlsx')

corpusA = df.sms.tolist()
labelA = df.label.tolist()
len(df)

1143

# Ekstraksi Fitur

In [4]:
fitur = lib.feature_extraction(corpusA)

# Seleksi Fitur Menggunaka Algoritma Genetika

In [5]:
jumlah_populasi = 200
jumlah_point = 1000 #jumlah titik point crossover
prob_mutasi = 0.2 #probabilitas permutasi

additional_child = True

K=25

metode = "tfidf"

alpha = 1
target = 0.99 #target akurasi

panjang_fitur = len(fitur)

features_1_bin = [1 for i in range(panjang_fitur)]
pop = lib.create_population(jumlah_populasi-1, panjang_fitur)
pop.append(features_1_bin)

fns_1 = lib.fitness_kf(corpusA, labelA, features_1_bin, fitur, alpha = alpha, metode = metode, K=K)
print("fitness all features:",fns_1)

fitnes_pop = list()
# temp_features_fitness = list()
for features_bin in pop:
    fns = lib.fitness_kf(corpusA, labelA, features_bin, fitur, alpha = alpha, metode = metode, K=K)
    fitnes_pop.append(fns)

print(len(fitnes_pop),"|",len(pop))

pop_used = dict()
popp_0 = list(pop)
for p, fit in zip(popp_0, fitnes_pop):
    bin_str = lib.int_to_str(p) 
    pop_used.update({bin_str:fit})

# print("")
    
# print(pop[0])

best_fitur_list = list()
best_fitness_list = list()
best_generasi_list = list()

best_jumlah_fitur_list = list()

gen=0
while True:
    gen+=1
    print("-"*25)
    print("Generasi ke",gen , "[Terbaik]")
    print("   "+"________________________________________")
    print("     Nilai Fitness       |fitur|ALL  |Generasi")
    rw = lib.get_roulette_wheel_(fitnes_pop, inc=jumlah_populasi)
    parents = lib.select_parents(rw)

    best_fitur_list_per_generasi = list()
    best_fitness_list_per_generasi = list()
    for p in parents:
        best_generasi_list.append(gen)
        family_fitness=list()
        family_fitur=list()

        mama_index = p[0]
        papa_index = p[1]
        
        m4ms = fitnes_pop[mama_index]
        family_fitness.append(m4ms)
        m4mr = list(pop[mama_index])
        family_fitur.append(m4mr)
        
        p4ps = fitnes_pop[papa_index]
        family_fitness.append(p4ps)
        p4pr = list(pop[papa_index])
        family_fitur.append(p4pr)
        
        #crossover dan mutasi
        bin_mama = list(pop[mama_index])
        bin_papa = list(pop[papa_index])
        anak_binary = lib.crossover(bin_mama, bin_papa, panjang_fitur,  jumlah_titik = jumlah_point, prob_mutasi = prob_mutasi)
        
        #mencari fitness untuk child
        a1 = anak_binary[0]
        a2 = anak_binary[1]
        a3 = anak_binary[2]
#         print(a2)
        
        if sum(a1)<=0:
            a1[-1]=1
            a1[-2]=1
        if sum(a2)<=0:
            a2[-1]=1
            a2[-2]=1
        if sum(a3)<=0:
            a3[-1]=1
            a3[-2]=1

        str_a1 = lib.int_to_str(a1)
        if str_a1 in pop_used:
            family_fitur.append(a1)
            family_fitness.append(pop_used[str_a1])
            
        else:
            family_fitur.append(a1)
            fitness_anak_1 = lib.fitness_kf(corpusA, labelA, a1, fitur, alpha = alpha, metode = metode, K=K)
            family_fitness.append(fitness_anak_1)
            pop_used.update({str_a1:fitness_anak_1})
            
        str_a2 = lib.int_to_str(a2)
        if str_a2 in pop_used:
            family_fitur.append(a2)
            family_fitness.append(pop_used[str_a2])
        else:
            family_fitur.append(a2)
            fitness_anak_2 = lib.fitness_kf(corpusA, labelA, a2, fitur, alpha = alpha, metode = metode, K=K)
            family_fitness.append(fitness_anak_2)
            pop_used.update({str_a2:fitness_anak_2})
        
        if additional_child == True:
            str_a3 = lib.int_to_str(a3)
            if str_a3 in pop_used:
                family_fitur.append(a3)
                family_fitness.append(pop_used[str_a3])
            else:
                family_fitur.append(a3)
                fitness_anak_3 = lib.fitness_kf(corpusA, labelA, a3, fitur, alpha = alpha, metode = metode, K=K)
                family_fitness.append(fitness_anak_3)
                pop_used.update({str_a3:fitness_anak_3})

        
        #mencari fitness terbaik untuk satu keluarga
        best_family_fitness = max(family_fitness)
        bf_index = family_fitness.index(best_family_fitness)
        best_family_fitur = family_fitur[bf_index]

        best_fitness_list.append(best_family_fitness)
        best_fitur_list.append(best_family_fitur)

        best_fitness_list_per_generasi.append(best_family_fitness)
        best_fitur_list_per_generasi.append(best_family_fitur)
        best_jumlah_fitur_list.append(sum(best_family_fitur))

#         print("   ",best_family_fitness)
        print("   ",lib.tampil_finess(best_family_fitness),"|",str(sum(best_family_fitur))+"|"+str(len(best_family_fitur)),"|",gen)

    pop = list(best_fitur_list_per_generasi)
#     pop.append(features_1_bin)
    fitnes_pop = list(best_fitness_list_per_generasi)
#     fitnes_pop.append(fns_1)

    if len(pop)<2 or max(best_fitness_list)>=target:
        print("="*70)
        print("Best",max(best_fitness_list))
#         print("Generasi ke-",best_generasi_list[best_fitness_list.index(max(best_fitness_list))])
        good_fitur = best_fitur_list[best_fitness_list.index(max(best_fitness_list))]
        best_fitness_list.index(max(best_fitness_list))
        
        best_fitur = np.array(fitur)[lib.get_index(best_fitness_list[-1])]
        best_score = best_fitness_list[-1]
        
        good_fitur__ = np.array(fitur)[lib.get_index(good_fitur)] 
        print('jumlah fitur     ', sum(good_fitur))
        print('jumlah fitur asli', panjang_fitur)
        print('good_fitur')
        
        dict_0919 = {
            'best_score':best_score,
            'best_fitur':best_fitur,
            'good_fitur':good_fitur__
        }
        
        break

fitness all features: 0.9342995169082127


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\lufias\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3325, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-407446ff661d>", line 26, in <module>
    fns = lib.fitness_kf(corpusA, labelA, features_bin, fitur, alpha = alpha, metode = metode, K=K)
  File "D:\github\python\Seleksi_fitur_Algen_MNB\Lib\Algen_lib.py", line 81, in fitness_kf
    skor_all.append(fitness_(X_train,y_train, X_test,y_test, binary_fitur, fitur, alpha = alpha, metode = metode))
  File "D:\github\python\Seleksi_fitur_Algen_MNB\Lib\Algen_lib.py", line 61, in fitness_
    data = weighting(X, metode = metode, feature=features[index])
  File "D:\github\python\Seleksi_fitur_Algen_MNB\Lib\Algen_lib.py", line 23, in weighting
    X = model.transform(data_)
  File "C:\Users\lufias\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py", line 1680, in transform
    X = super().transform(raw_documents)
  F

KeyboardInterrupt: 

In [5]:
#Testing
hasil_seleksi_fitur = good_fitur
lib.fitness_kf(corpusA, labelA, hasil_seleksi_fitur, fitur, alpha = alpha, metode = metode, K=K)

0.9387053140096618

In [6]:
# for i in np.array(fitur)[lib.get_index(hasil_seleksi_fitur)]:
#     print(i, end=", ")

In [7]:
import json
fitur_pilihan = np.array(fitur)[lib.get_index(good_fitur)]
print(len(fitur_pilihan))
fitur_pilihan

dict_dasddas = {
    "score":max(best_fitness_list),
    "fitur":fitur_pilihan.tolist(),
    "K":K
}

with open("fitur/fitur_p3.json","w") as f:
    json.dump(dict_dasddas,f)

3731


In [8]:

dict_data = {
    "fitness":best_fitness_list,
    "jumlah_fitur":best_jumlah_fitur_list,
    "generasi":best_generasi_list,
}

df_hasil = pd.DataFrame.from_dict(dict_data)
df_hasil


,fitness,jumlah_fitur,generasi
0,0.918609,3357,1
1,0.924696,3281,1
2,0.925585,3253,1
3,0.926531,3248,1
4,0.926435,3291,1
...,...,...,...
197,0.937816,3734,6
198,0.938705,3731,6
199,0.937836,4315,7
200,0.938705,3731,7


In [9]:
print(len(best_fitness_list))

202


In [10]:
cek = 0.937

grouped = df_hasil.groupby('generasi')
grouped.filter(lambda x: x['fitness'].mean() > cek)

,fitness,jumlah_fitur,generasi
188,0.937816,3734,5
189,0.937836,4315,5
190,0.938686,4314,5
191,0.937816,4349,5
192,0.936947,4331,5
193,0.935188,4375,5
194,0.937836,4110,5
195,0.938686,4314,6
196,0.937836,4315,6
197,0.937816,3734,6


In [6]:
jumlah_populasi = 200
jumlah_point = 1000 #jumlah titik point crossover
prob_mutasi = 0.2 #probabilitas permutasi

additional_child = True

K=25

metode = "tfidf"

alpha = 1
target = 0.99 #target akurasi

panjang_fitur = len(fitur)



In [7]:
hasil_seleksi = features_selection(pop=pop, jumlah_populasi=jumlah_populasi,
                                   jumlah_titik=jumlah_point, prob_mutasi=prob_mutasi,
                                   K=K, alpha=alpha, target=target, 
                                   panjang_fitur=panjang_fitur, additional_child=True)

fitness all features: 0.9342995169082127
200 | 200
-------------------------
Generasi ke 1 [Terbaik]
   ________________________________________
     Nilai Fitness       |fitur|ALL  |Generasi
    0.925584541062802    | 3248|4378 | 1
    0.9239033816425121   | 3316|4378 | 1
    0.9264927536231885   | 2174|4378 | 1
    0.9229758454106282   | 3236|4378 | 1
    0.923033816425121    | 2231|4378 | 1
    0.9239033816425122   | 3287|4378 | 1
    0.9316908212560387   | 3274|4378 | 1
    0.9229951690821256   | 3263|4378 | 1
    0.9151304347826087   | 3228|4378 | 1
    0.9291014492753624   | 3238|4378 | 1
    0.9229951690821256   | 3301|4378 | 1
    0.9247342995169083   | 3289|4378 | 1
    0.9256425120772949   | 3288|4378 | 1
    0.9291014492753624   | 3309|4378 | 1
    0.9255458937198068   | 3311|4378 | 1
    0.9212753623188407   | 3295|4378 | 1
    0.9282512077294686   | 3256|4378 | 1
    0.9291594202898551   | 3271|4378 | 1
    0.9290048309178744   | 3252|4378 | 1
    0.922937198067633    | 32

    0.9360966183574879   | 4108|4378 | 4
    0.9387246376811595   | 3838|4378 | 4
    0.9369661835748794   | 4305|4378 | 4
-------------------------
Generasi ke 5 [Terbaik]
   ________________________________________
     Nilai Fitness       |fitur|ALL  |Generasi
    0.9361159420289856   | 3779|4378 | 5
    0.937816425120773    | 4099|4378 | 5
    0.9388019323671499   | 3275|4378 | 5
    0.9387246376811595   | 3838|4378 | 5
    0.9369661835748794   | 4305|4378 | 5
    0.9369661835748794   | 3689|4378 | 5
    0.937816425120773    | 4099|4378 | 5
-------------------------
Generasi ke 6 [Terbaik]
   ________________________________________
     Nilai Fitness       |fitur|ALL  |Generasi
    0.9369661835748794   | 3689|4378 | 6
    0.9388019323671499   | 3275|4378 | 6
    0.937816425120773    | 4099|4378 | 6
    0.937816425120773    | 4099|4378 | 6
-------------------------
Generasi ke 7 [Terbaik]
   ________________________________________
     Nilai Fitness       |fitur|ALL  |Generasi
   

TypeError: 'numpy.float64' object is not iterable